# Sync Link
### Part 5: Application

In [39]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import pickle
import musicbrainzngs

In [2]:
title = 'Tennessee Whiskey'
artist = 'Chris Stapleton'
search_test = title.lower() + ' ' + artist.lower()

In [3]:
search_test

'tennessee whiskey chris stapleton'

### Deezer (Master)

In [4]:
deez_url = f'https://api.deezer.com/search?q=artist:\"{artist}\" track:\"{title}\"'
deezer_q = requests.get(deez_url)
track = str(deezer_q.json()['data'][0]['id'])

deezer_track = requests.get('https://api.deezer.com/track/' + track)
data = deezer_track.json()

In [5]:
al = str(data['album']['id'])
deezer_album = requests.get('https://api.deezer.com/album/' + al)
album = deezer_album.json()
album['upc']
album['label']

'Mercury Nashville'

In [6]:
def gather_deezer(song_s, artist_s):
        master_info = {}
        try:
            
            deez_url = f'https://api.deezer.com/search?q=artist:\"{artist_s}\" track:\"{song_s}\"'
            deezer_q = requests.get(deez_url)
            track = str(deezer_q.json()['data'][0]['id'])

            deezer_track = requests.get('https://api.deezer.com/track/' + track)
            data = deezer_track.json()

            
            master_info['Master'] = 'Master Info from Deezer'
            master_info['Recording Title'] = data['title']
            master_info['Artist'] = data['artist']['name']
            master_info['Album Title'] = data['album']['title']
            master_info['ISRC'] =  data['isrc']
            master_info['Release Date'] = data['release_date']
            master_info['Is Explicit?']= data['explicit_lyrics']
            master_info['BPM'] = data['bpm']
            master_info['Deezer Track ID'] = data['id']
            al = str(data['album']['id'])
            deezer_album = requests.get('https://api.deezer.com/album/' + al)
            album = deezer_album.json()
            master_info['Album UPC'] = album['upc']
            master_info['Label'] = album['label']
            master_info['Link to Album on Deezer'] = data['album']['link']
            
           
            master_info['Album Art']= data['album']['cover_medium']
            
           
        
        except:
            print(f'Could not find on Deezer.')
    
        master_df = pd.DataFrame([master_info])
        
        master_df.set_index('Master', inplace = True)
       
        
        return master_df.T

In [7]:
master = gather_deezer(title, artist)

In [8]:
master

Master,Master Info from Deezer
Recording Title,Tennessee Whiskey
Artist,Chris Stapleton
Album Title,Traveller
ISRC,USUM71418088
Release Date,2015-05-04
Is Explicit?,False
BPM,152
Deezer Track ID,98975170
Album UPC,602547273123
Label,Mercury Nashville


In [9]:
title = master.iloc[0, 0].lower()
artist = master.iloc[1, 0].lower()
search_new = title + ' ' + artist

In [10]:
search_new

'tennessee whiskey chris stapleton'

### Musixmatch (Publishing)

In [11]:
from credentials import mx_key

In [33]:
mx_track_search = 'https://api.musixmatch.com/ws/1.1/matcher.lyrics.get?track_isrc=USUM71418088'

params = {
            'apikey' : mx_key,
#             'q_track' : 'Don\'t Start Now',
#              'q_artist' : 'Dua Lipa',
#              's_track_release_date' : 'ASC'
                }

res = requests.get(mx_track_search, params)


In [43]:
musicbrainzngs.set_useragent('sync_link', '1.0', contact='synclink.io')

In [143]:
result = musicbrainzngs.get_recordings_by_isrc('USUM71418088')

In [149]:
result['isrc']['recording-list'][0]['id']

'a6f69eb9-da2e-4b4e-ab78-69fa9f75d6b6'

In [156]:
result = musicbrainzngs.get_recording_by_id('a6f69eb9-da2e-4b4e-ab78-69fa9f75d6b6', includes=['work-level-rels', 'work-rels'],)





In [157]:
result['recording']['work-relation-list'][0]['work']['id']

'f49f4178-e697-45a1-b62e-16fba93fb0cc'

In [160]:
result['recording']['title']

'Tennessee Whiskey'

In [150]:
result = musicbrainzngs.get_work_by_id('f49f4178-e697-45a1-b62e-16fba93fb0cc',
                                      includes=['annotation', 'aliases', 
                                               'artist-rels', 'label-rels',  'work-rels'])


In [151]:
writer = []
for i in result['work']['artist-relation-list']:
    if i['type'] == 'writer':
        writer.append(i['artist']['name'])

In [152]:
writer

['Dean Dillon', 'Linda Hargrove']

In [153]:
pub = []
for i in result['work']['label-relation-list']:
    if i['type'] == 'publishing':
        pub.append(i['label']['name'])

In [154]:
pub

['EMI Algee Music Corp.', 'Universal‐Songs of PolyGram International, Inc.']

In [131]:
result

{'work': {'id': 'f49f4178-e697-45a1-b62e-16fba93fb0cc',
  'type': 'Song',
  'title': 'Tennessee Whiskey',
  'language': 'eng',
  'attribute-list': [{'attribute': 'BMI ID', 'value': '1472421'}],
  'artist-relation-list': [{'type': 'writer',
    'type-id': 'a255bca1-b157-4518-9108-7b147dc3fc68',
    'target': 'fdb4cb37-d38c-49bb-afad-485cbed17950',
    'direction': 'backward',
    'artist': {'id': 'fdb4cb37-d38c-49bb-afad-485cbed17950',
     'type': 'Person',
     'name': 'Dean Dillon',
     'sort-name': 'Dillon, Dean'},
    'target-credit': 'Dean Rutherford'},
   {'type': 'writer',
    'type-id': 'a255bca1-b157-4518-9108-7b147dc3fc68',
    'target': '0ac020b1-5763-4310-9849-d5d531c25753',
    'direction': 'backward',
    'artist': {'id': '0ac020b1-5763-4310-9849-d5d531c25753',
     'type': 'Person',
     'name': 'Linda Hargrove',
     'sort-name': 'Hargrove, Linda'}}],
  'label-relation-list': [{'type': 'publishing',
    'type-id': '05ee6f18-4517-342d-afdf-5897f64276e3',
    'target': '

In [134]:
res = requests.get('https://musicbrainz.org/ws/2/work/f49f4178-e697-45a1-b62e-16fba93fb0cc?fmt=json')

In [137]:
res.json()['iswcs']

[]

### Lyric Freak (Publishing)

In [37]:
def clean_title(song):
    song = song.replace('About', '').replace('Lyrics', '').replace('lyrics', '')
    try:
        song = song.split('–')[1]
    except:
        pass
    song = song.strip()
    return song

In [38]:
def clean_pub(string):
    try:
        string = string.split('©')[1]
        string = string.split('Lyrics')[0]
        if ', Inc.' in string:
            string = string.replace(', Inc.', '')
        if '\\n' in string:
            string = string.replace('\\n', '')  
    except:
        pass
    return string.strip()

In [39]:
def gather_lyricfreak(search):
    pub_info = {}
    
    try:
        base = 'https://www.lyricsfreak.com/search.php?q='
        query = search.replace('-', '').replace(' ', '%20')
        res = requests.get(base + query)
        soup = BeautifulSoup(res.content, 'lxml')
        song = soup.find('a', {'class': 'song'})
        url = song['href']
    
        res_2 = requests.get('https://www.lyricsfreak.com' + url)
        soup_2 = BeautifulSoup(res_2.content, 'lxml')
    
        title = soup_2.find('h2').text
        artist = soup_2.find('a', {'class' : 'song-page-conthead-link'}).text


        writer_pub = soup_2.find_all('div', {'class' : 'meta_l'})
        writer = writer_pub[0].text
        pub = writer_pub[1].text

        if  soup_2.find('div', {'class' : 'lf-hero__subtitle'}):
            album = soup_2.find('div', {'class' : 'lf-hero__subtitle'}).text 
            pub_info['Album'] = album.split('album:\n')[1].strip().split(' ')[0]

        #Putting it into the dictionary
        pub_info['Publishing'] = 'Publishing Info from Lyric Freak'
        pub_info['Composition Title'] = clean_title(title)
        pub_info['Performed By'] = artist
        pub_info['Writers'] = writer.replace('Songwriters: ', '')
        pub_info['Publishers'] = clean_pub(pub)
        
        pub_df = pd.DataFrame([pub_info])
        
        pub_df.set_index('Publishing', inplace = True)
        
        return pub_df.T
    
    except:
        return f'Could not find on Lyric Freak.' 

In [40]:
publishing = gather_lyricfreak(search_new)

In [41]:
publishing

Publishing,Publishing Info from Lyric Freak
Album,Traveller
Composition Title,Tennessee Whiskey
Performed By,Chris Stapleton
Writers,"Linda H Bartholomew, Dean Dillon"
Publishers,"Sony/ATV Music Publishing LLC, Universal Music..."


### Genius (Lyrics)

In [42]:
def gather_genius(search):
    from credentials import genius
    header = {'Authorization' : f'Bearer {genius}'}
    
    res = requests.get('https://api.genius.com/search?q=' + search, headers = header)
    
    
    song = res.json()['response']['hits'][0]['result']['api_path']
    res2 = requests.get('https://api.genius.com' + song, headers = header)
    embed = res2.json()['response']['song']['embed_content']
    
    return embed

In [43]:
genius_lyrics = gather_genius(search_new)

### Spotify (Song Features)

In [45]:
def auth(key, sec):
    
    authorize = 'https://accounts.spotify.com/api/token'
    param = {
    "Content-Type": "application/x-www-form-urlencoded",
    'grant_type' : 'client_credentials'
    }
    res = requests.post(authorize, auth = (s_key, ss_key), data = param)
    token = res.json()['access_token']
    
    return token

In [46]:
token = auth(s_key, ss_key)    

In [129]:
def get_spotify(search):
    spotify_info = {}
    
    token = auth(s_key, ss_key)
    
    try:

        #Search parameters with the song plugged in
        params = {
        'q' : search,
        'type': 'track',
        'limit' : 5
        }

        #Header for authorization
        header = {'Authorization' : f'Bearer {token}'}

        #Search endpoint
        spotify_search = 'https://api.spotify.com/v1/search'

        #Make the request
        res = requests.get(spotify_search, headers = header, params = params)
        status = res.status_code

        results = res.json()

        spotify_info['Artist'] = results['tracks']['items'][0]['artists'][0]['name']
        spotify_info['Song Title'] = results['tracks']['items'][0]['name']
        uri = results['tracks']['items'][0]['id']
        album_id = results['tracks']['items'][0]['album']['id']
        embed = f'<iframe src="https://open.spotify.com/embed/album/{album_id}" width="300" height="380" frameborder="0" allowtransparency="true" allow="encrypted-media"></iframe>'
    
        res_2 = requests.get(f'https://api.spotify.com/v1/audio-features/{uri}', headers = header) 
    
        results_2 = res_2.json()
    
        spotify_info['Spotify'] = 'Spotify Audio Features'
        spotify_info['Danceeability'] = results_2['danceability']
        spotify_info['Energy'] = results_2['energy']
        spotify_info['Key'] = results_2['key']
        spotify_info['Loudness'] = results_2['loudness']
        spotify_info['Mode'] = results_2['mode']
        spotify_info['Speechiness'] = results_2['speechiness']
        spotify_info['Acousticness'] = results_2['acousticness']
        spotify_info['Instrumentalness'] = results_2['instrumentalness']
        spotify_info['Liveness'] = results_2['liveness']
        spotify_info['Valence'] = results_2['valence']
        spotify_info['Tempo'] = results_2['tempo']
        spotify_info['Time Signature'] = results_2['time_signature']
    
    except:
        return 'Could not find on Spotify.'
            
    spotify_df = pd.DataFrame([spotify_info])
        
    spotify_df.set_index('Spotify', inplace = True)

    return spotify_df.T, embed

In [130]:
params = {
'q' : search_new,
'type': 'track',
'limit' : 5
}

#Header for authorization
header = {'Authorization' : f'Bearer {token}'}

#Search endpoint
spotify_search = 'https://api.spotify.com/v1/search'

#Make the request
res = requests.get(spotify_search, headers = header, params = params)
status = res.status_code

results = res.json()

In [117]:
results['tracks']['items'][0]['album']['images'][1]['url']

'https://i.scdn.co/image/ab67616d00001e02540fc1d083eac5bcff8dad21'

In [ ]:
spotify_info['Song Title'] = 
        spotify_info['Artist'] = 
        spotify_info['Album'] = 
        spotify_info['ISRC'] = 
        spotify_info['Release Date'] = 
        spotify['Is Explicit?'] = results['tracks']['items'][0]['explicit']
        spotify['Spotify URI'] = results['tracks']['items'][0]['id']
        uri = results['tracks']['items'][0]['id']
        album_id = results['tracks']['items'][0]['album']['id']

In [131]:
def spotify_master(search):
    spotify_info = {}
    
    token = auth(s_key, ss_key)
    
    try:

        #Search parameters with the song plugged in
        params = {
        'q' : search,
        'type': 'track',
        'limit' : 5
        }
        
        
        #Header for authorization
        header = {'Authorization' : f'Bearer {token}'}

        #Search endpoint
        spotify_search = 'https://api.spotify.com/v1/search'

        #Make the request
        res = requests.get(spotify_search, headers = header, params = params)
        status = res.status_code

        results = res.json()
        
        spotify_info['Song Title'] = results['tracks']['items'][0]['name']
        spotify_info['Artist'] = results['tracks']['items'][0]['artists'][0]['name']
        spotify_info['Album'] = results['tracks']['items'][0]['album']['name']
        spotify_info['ISRC'] = results['tracks']['items'][0]['external_ids']['isrc']
        spotify_info['Release Date'] = results['tracks']['items'][0]['album']['release_date']
        spotify_info['Is Explicit?'] = results['tracks']['items'][0]['explicit']
        spotify_info['Spotify URI'] = results['tracks']['items'][0]['id']
        uri = results['tracks']['items'][0]['id']
        album_id = results['tracks']['items'][0]['album']['id']
        embed = f'<iframe src="https://open.spotify.com/embed/album/{album_id}" width="300" height="380" frameborder="0" allowtransparency="true" allow="encrypted-media"></iframe>'
        art = results['tracks']['items'][0]['album']['images'][1]['url']
        
        
        res_2 = requests.get(f'https://api.spotify.com/v1/albums/{album_id}', headers = header) 
        
        results_2 = res_2.json()
        spotify_info['Album UPC'] = results_2['external_ids']['upc']
        spotify_info['Label'] = results_2['label']
        
    
        spotify_info['Spotify'] = 'Master Info from Spotify'
    except:
        return 'Could not find on Spotify.', 'None'
            
    spotify_df = pd.DataFrame([spotify_info])
        
    spotify_df.set_index('Spotify', inplace = True)

    
    return spotify_df.T, embed, art

In [132]:
mast, embed, art = spotify_master(search_new)

In [133]:
mast

Spotify,Master Info from Spotify
Song Title,Tennessee Whiskey
Artist,Chris Stapleton
Album,Traveller
ISRC,USUM71418088
Release Date,2015-05-04
Is Explicit?,False
Spotify URI,3fqwjXwUGN6vbzIwvyFMhx
Album UPC,00602547273123
Label,Mercury Nashville


In [89]:
res['external_ids']['upc']

'00602547273123'

In [58]:
#Artwork
res['images'][2]

{'height': 64,
 'url': 'https://i.scdn.co/image/ab67616d00004851540fc1d083eac5bcff8dad21',
 'width': 64}

In [104]:
spotify, embed = get_spotify(search_new)

In [105]:
embed

'<iframe src="https://open.spotify.com/embed/album/7lxHnls3yQNl8B9bILmHj7" width="300" height="380" frameborder="0" allowtransparency="true" allow="encrypted-media"></iframe>'

### Sync Score

In [106]:
cvec = pickle.load(open('./model/cvec.pkl', 'rb'))
rf = pickle.load(open('./model/rf.pkl', 'rb'))

In [107]:
features = ['year', 'explicit', 
       'n_writers', 'n_pub', 
       's_dance', 's_energy', 's_key', 's_loudness', 's_mode',
       's_speech', 's_acoustic', 's_inst', 's_live', 's_valence', 's_tempo',
       's_time_sig']

In [161]:
pred_df = {
    'year' : master.iloc[4, 0][:4],
    'explicit' : int(master.iloc[5, 0]),
    'n_writers' : writers,
    'n_pub' : publishers,
    's_dance' : audio.iloc[2, 0],
    's_energy' : audio.iloc[3, 0],
    's_key' : audio.iloc[4, 0],
    's_loudness' : audio.iloc[5, 0],
    's_mode' : audio.iloc[6, 0],
    's_speech' : audio.iloc[7, 0],
    's_acoustic' : audio.iloc[8, 0],
    's_inst' : audio.iloc[9, 0],
    's_live': audio.iloc[10, 0],
    's_valence' : audio.iloc[11, 0],
    's_tempo': audio.iloc[12, 0],
    's_time_sig' : audio.iloc[13, 0]
}


SyntaxError: invalid syntax (<ipython-input-161-7ed32208b7d5>, line 5)

In [109]:
pred_df['s_dance'] = spotify.iloc[2, 0]
pred_df['s_energy'] = 
pred_df['s_key']= 
pred_df['s_loudness']= 
pred_df['s_mode']= 
pred_df['s_speech']= 
pred_df['s_acoustic']= 
pred_df['s_inst']= 
pred_df['s_live']= 
pred_df['s_valence'] = 
pred_df['s_tempo'] = 
pred_df['s_time_sig'] = 

In [110]:
publishing

Publishing,Publishing Info from Lyric Freak
Album,Traveller
Composition Title,Tennessee Whiskey
Performed By,Chris Stapleton
Writers,"Linda H Bartholomew, Dean Dillon"
Publishers,"Sony/ATV Music Publishing LLC, Universal Music..."


In [111]:
pred_df['text'] = master.iloc[1, 0].lower() + " " + publishing.iloc[2, 0] + " " + publishing.iloc[3, 0]

In [112]:
pred_df['text']

'chris stapleton Chris Stapleton Linda H Bartholomew, Dean Dillon'

In [113]:
to_predict = pd.DataFrame([pred_df])

In [114]:
def sync_score(mast, pub, spot):
    cvec = pickle.load(open('./model/cvec.pkl', 'rb'))
    rf = pickle.load(open('./model/rf.pkl', 'rb'))
    
    
    pred_df = {}
    pred_df['year'] = mast.iloc[4, 0][:4]
    pred_df['explicit'] = int(mast.iloc[5, 0])
    pred_df['n_writers'] = len(pub.iloc[2, 0].split(','))
    pred_df['n_pub'] = len(pub.iloc[3, 0].split(','))

    
    pred_df['s_dance'] = spot.iloc[2, 0]
    pred_df['s_energy'] = spot.iloc[3, 0]
    pred_df['s_key']= spot.iloc[4, 0]
    pred_df['s_loudness']= spot.iloc[5, 0]
    pred_df['s_mode']= spot.iloc[6, 0]
    pred_df['s_speech']= spot.iloc[7, 0]
    pred_df['s_acoustic']= spot.iloc[8, 0]
    pred_df['s_inst']= spot.iloc[9, 0]
    pred_df['s_live']= spot.iloc[10, 0]
    pred_df['s_valence'] = spot.iloc[11, 0]
    pred_df['s_tempo'] = spot.iloc[12, 0]
    pred_df['s_time_sig'] = spot.iloc[13, 0]
    
    pred_df['text'] = mast.iloc[1, 0].lower() + " " + pub.iloc[2, 0] + " " + pub.iloc[3, 0]
    
    to_predict = pd.DataFrame([pred_df])
    vec = cvec.transform(to_predict['text'])
    x_text = pd.DataFrame(vec.toarray(),
                 columns=cvec.get_feature_names())
    X = pd.concat([x_text, to_predict[features]], axis = 1)
    score =  rf.predict_proba(X)[0][1]
    
    if score >= .50:
        return 'Syncable'
    
    else:
        return 'Not very syncable'

In [115]:
test = 'not'

In [116]:
type(test) == str

True

In [137]:
def get_all(artist, title):

    try:
        master = gather_deezer(title, artist)
        title = master.iloc[0, 0].lower()
        artist = master.iloc[1, 0].lower()
        title_display = master.iloc[0, 0]
        artist_display = master.iloc[1, 0]
        search_new = title + ' ' + artist
        artwork = master.iloc[11, 0]

        genius_lyrics = gather_genius(search_new)

        publishing = gather_lyricfreak(search_new)

        spotify, embed = get_spotify(search_new)

        if (type(publishing != str)) and (type(spotify != str)):
            score = sync_score(master, publishing, spotify)
        else:
            score = 'Not enough info for Sync Score'
        if master.iloc[5, 0] == False:
            master.iloc[5, 0] = 'No'
        else:
            master.iloc[5, 0] = 'Yes'

    except:
        try:
            search_new = artist + ' ' + title
            master, embed, art = spotify_master(search_new)
            print(master)
            title = master.iloc[0, 0].lower()
            artist = master.iloc[1, 0].lower()
            title_display = master.iloc[0, 0]
            artist_display = master.iloc[1, 0]

            genius_lyrics = gather_genius(search_new)

            publishing = gather_lyricfreak(search_new)

            spotify, embed = get_spotify(search_new)

            if (type(publishing != str)) and (type(spotify != str)):
                score = sync_score(master, publishing, spotify)
            else:
                score = 'Not enough info for Sync Score'
            if master.iloc[5, 0] == False:
                master.iloc[5, 0] = 'No'
            else:
                master.iloc[5, 0] = 'Yes'


        except:
            title_display = "None"
            artist_display = "None"
            master =  "None"
            publishing = "None"
            genius_lyrics = "None"
            spotify = "None"
            score = "None"
            embed = "None"
            artwork = 'None'

    

    return art, title_display, artist_display, master, publishing, genius_lyrics, spotify, score, embed





In [138]:
%%time
artwork, title_display, artist_display, master, publishing, genius_lyrics, spotify, score, embed = get_all('Emily', 'Clen Cut Kid')

Could not find on Deezer.
Spotify      Master Info from Spotify
Song Title                      Emily
Artist                  Clean Cut Kid
Album                        Painwave
ISRC                     GBKPL1820051
Release Date               2019-03-29
Is Explicit?                    False
Spotify URI    3m7QGY2wCCAjNIfsnFdqfW
Album UPC               5054526891937
Label             Babe Magnet Records
CPU times: user 230 ms, sys: 15.8 ms, total: 246 ms
Wall time: 5.14 s


In [136]:
master

'None'

In [144]:
genius_lyrics

"<div id='rg_embed_link_4539729' class='rg_embed_link' data-song-id='4539729'>Read <a href='https://genius.com/Tones-and-i-dance-monkey-lyrics'>“Dance Monkey” by Tones\xa0and I</a> on Genius</div> <script crossorigin src='//genius.com/songs/4539729/embed.js'></script>"

In [145]:
embed

'<iframe src="https://open.spotify.com/embed/album/0UywfDKYlyiu1b38DRrzYD" width="300" height="380" frameborder="0" allowtransparency="true" allow="encrypted-media"></iframe>'

In [146]:
score

'Syncable'

In [147]:
explicit_d = {False : 'No', True : 'Yes'}

In [153]:
master.iloc[5][0].map(explicit_d)

AttributeError: 'bool' object has no attribute 'map'

In [149]:
master.iloc[5] = master.iloc[5, 0].map(explicit_d)

AttributeError: 'bool' object has no attribute 'map'

In [154]:
master.iloc[5, 0]

False

In [122]:
master.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th>Master</th>\n      <th>Master Info from Deezer</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Recording Title</th>\n      <td>Dance Monkey</td>\n    </tr>\n    <tr>\n      <th>Artist</th>\n      <td>Tones and I</td>\n    </tr>\n    <tr>\n      <th>Album Title</th>\n      <td>The Kids Are Coming</td>\n    </tr>\n    <tr>\n      <th>ISRC</th>\n      <td>QZES71982312</td>\n    </tr>\n    <tr>\n      <th>Release Date</th>\n      <td>2019-08-29</td>\n    </tr>\n    <tr>\n      <th>Is Explicit?</th>\n      <td>False</td>\n    </tr>\n    <tr>\n      <th>BPM</th>\n      <td>97.97</td>\n    </tr>\n    <tr>\n      <th>Deezer Track ID</th>\n      <td>739870792</td>\n    </tr>\n    <tr>\n      <th>Link to Album on Deezer</th>\n      <td>https://www.deezer.com/album/108770322</td>\n    </tr>\n    <tr>\n      <th>Album Art</th>\n      <td>https://cdns-images.dzcdn.net/images/cover/5636a56973

In [123]:
publishing.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th>Publishing</th>\n      <th>Publishing Info from Lyric Freak</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Composition Title</th>\n      <td>Dance Monkey</td>\n    </tr>\n    <tr>\n      <th>Performed By</th>\n      <td>Tones And I</td>\n    </tr>\n    <tr>\n      <th>Writers</th>\n      <td>Toni Watson</td>\n    </tr>\n    <tr>\n      <th>Publishers</th>\n      <td>Warner Chappell Music</td>\n    </tr>\n  </tbody>\n</table>'